In [1]:
import os

import requests
from bs4 import BeautifulSoup

In [2]:
def read_credentials():
    creds = dict()
    creds['YRASP_APIKEY'] = os.environ['YRASP_APIKEY']
    return creds


# API синтаксис

https://yandex.ru/dev/rasp/doc/reference/schedule-point-point-docpage/

## расписание от и до
```
& from=<код станции отправления> (обязательный)
& to=<код станции прибытия> (обязательный)
& [apikey=<ключ>] (обязательный)
& [format=<формат — XML или JSON>]
& [lang=<язык>]
& [date=<дата>]
& [transport_types=<тип транспорта>]
& [system=<система кодирования параметров to и from>]
& [show_systems=<система кодирования для ответа>]
& [offset=<сдвиг относительно первого рейса в ответе>]
& [limit=<ограничение на количество рейсов в ответе>]
& [add_days_mask=<запрос календаря хождения рейсов>]
& [result_timezone=<часовой пояс>]
& [transfers=<признак запроса маршрутов с пересадками>]
```


## список станций

https://api.rasp.yandex.net/v3.0/stations_list/ ?
  apikey=<ключ>
[& format=<формат>]
[& lang=<язык>]

In [3]:
with open('points_of_interest.xml') as f:
    soup = BeautifulSoup(f.read(), 'xml')

In [4]:
for station in soup.points_of_interest.find_all('station'):
    print(station)
    print(station.title.text, station.yandex_code.text)

<station><title>Тестовская</title><longitude>37.531472</longitude><latitude>55.753827</latitude><transport_type>train</transport_type><station_type>platform</station_type><direction>Белорусское</direction><codes><esr_code>198226</esr_code><yandex_code>s9601349</yandex_code></codes></station>
Тестовская s9601349
<station><title>Савёловская</title><longitude>37.590095</longitude><latitude>55.79427</latitude><transport_type>train</transport_type><station_type>platform</station_type><direction>Курское</direction><codes><esr_code>191655</esr_code><yandex_code>s9601382</yandex_code></codes></station>
Савёловская s9601382
<station><title>Станколит</title><longitude>37.598675</longitude><latitude>55.799496</latitude><transport_type>train</transport_type><station_type>platform</station_type><direction>Курское</direction><codes><esr_code>191640</esr_code><yandex_code>s9601384</yandex_code></codes></station>
Станколит s9601384
<station><title>Дмитровская</title><longitude>37.578562</longitude><la

In [5]:
station_codes = {station.title.text:station.yandex_code.text for 
                 station in soup.points_of_interest.find_all('station')
                }

station_codes

{'Тестовская': 's9601349',
 'Савёловская': 's9601382',
 'Станколит': 's9601384',
 'Дмитровская': 's9600901',
 'Царицыно': 's9600891'}

In [6]:
station_codes_backup = {
 'Тестовская': 's9601349',
 'Савёловская': 's9601382',
 'Станколит': 's9601384',
 'Дмитровская': 's9600901',
     'Царицыно': 's9600891',

}

In [7]:
import json

In [8]:
import pandas as pd

In [9]:
api_creds = read_credentials()

def fetch_timetable(from_, to_, date) -> dict:
    """
    from_, to_ are station names
    
    date (str) should be a string in ISO format (YYYY-MM-DD)
    
    """
    assert from_ in station_codes
    assert to_ in station_codes

    r = requests.get(
        'https://api.rasp.yandex.net/v3.0/search',

        params={
            'from':station_codes[from_], 
              'to':station_codes[to_], 
          'format':'json',
            'date':date,
            'transport_types':'suburban', # электрички
            'transfers':False
                },

        headers={       'Accept': 'application/json',
                 'Authorization': api_creds['YRASP_APIKEY'],
                },
    )
    
    r_str = r.content.decode('utf-8')
    
    return json.loads(r_str)

In [10]:
def train_to_record(t):
    return {
         "отходит" : pd.to_datetime(t['departure']).tz_localize(None).tz_localize('EUROPE/MOSCOW'),
        "приходит" : pd.to_datetime(t['arrival']).tz_localize(None).tz_localize('EUROPE/MOSCOW'),
              "от" : t['from']['title'],
              "до" : t['to']['title'],
            "идёт" : f"{t['duration'] / 60:.0f} мин",
           "номер" : t['thread']['number'],
             "тип" : t['thread']['transport_subtype']['title'],
        }

In [11]:

def show_timetable(from_, to_):

    now = pd.Timestamp.now().tz_localize('EUROPE/MOSCOW')

    date = now.strftime('%Y-%m-%d')

    resp = fetch_timetable(from_, to_, date)
    trains = resp['segments']

    # filter out departures from the past
    trains = [t for t in trains if pd.Timestamp(t['departure']) > now]

    timetable_for_today = pd.DataFrame.from_records([train_to_record(t) for t in trains])
    timetable_for_today['через'] = pd.Series(((pd.TimedeltaIndex(timetable_for_today['отходит'] - now)
                                    .round('1min')
                                    .total_seconds()) / 60)).apply(lambda x: f"{x:.0f} мин")

    timetable_for_today['отходит в' ]=timetable_for_today['отходит'].dt.strftime('%Y-%m-%d %H:%M')
    timetable_for_today['приходит в']=timetable_for_today['приходит'].dt.strftime('%Y-%m-%d %H:%M')

    cols = ['отходит в', 'через', 'от', "до", 'идёт', 'тип', "приходит в"]
    return timetable_for_today[cols]

In [12]:
show_timetable('Тестовская', 'Станколит')

,отходит в,через,от,до,идёт,тип,приходит в
0,2019-10-25 15:08,40 мин,Тестовская,Станколит,17 мин,стандарт плюс,2019-10-25 15:25
1,2019-10-25 16:03,95 мин,Тестовская,Станколит,28 мин,Пригородный поезд,2019-10-25 16:31
2,2019-10-25 17:11,163 мин,Тестовская,Станколит,18 мин,стандарт плюс,2019-10-25 17:29
3,2019-10-25 17:26,178 мин,Тестовская,Станколит,17 мин,стандарт плюс,2019-10-25 17:43
4,2019-10-25 17:46,198 мин,Тестовская,Станколит,15 мин,Пригородный поезд,2019-10-25 18:01
5,2019-10-25 18:31,243 мин,Тестовская,Станколит,16 мин,стандарт плюс,2019-10-25 18:47
6,2019-10-25 18:43,255 мин,Тестовская,Станколит,19 мин,стандарт плюс,2019-10-25 19:02
7,2019-10-25 19:15,287 мин,Тестовская,Станколит,18 мин,Пригородный поезд,2019-10-25 19:33
8,2019-10-25 20:37,369 мин,Тестовская,Станколит,15 мин,стандарт плюс,2019-10-25 20:52
9,2019-10-25 22:13,465 мин,Тестовская,Станколит,16 мин,стандарт плюс,2019-10-25 22:29


In [13]:
show_timetable('Дмитровская', 'Царицыно')

,отходит в,через,от,до,идёт,тип,приходит в
0,2019-10-25 14:50,22 мин,Дмитровская,Царицыно,50 мин,стандарт плюс,2019-10-25 15:40
1,2019-10-25 14:55,27 мин,Дмитровская,Царицыно,50 мин,стандарт плюс,2019-10-25 15:45
2,2019-10-25 15:52,84 мин,Дмитровская,Царицыно,48 мин,стандарт плюс,2019-10-25 16:40
3,2019-10-25 16:22,114 мин,Дмитровская,Царицыно,46 мин,стандарт плюс,2019-10-25 17:08
4,2019-10-25 18:51,263 мин,Дмитровская,Царицыно,53 мин,стандарт плюс,2019-10-25 19:44
5,2019-10-25 18:57,269 мин,Дмитровская,Царицыно,52 мин,стандарт плюс,2019-10-25 19:49
6,2019-10-25 19:48,320 мин,Дмитровская,Царицыно,51 мин,стандарт плюс,2019-10-25 20:39
7,2019-10-25 20:12,344 мин,Дмитровская,Царицыно,50 мин,стандарт плюс,2019-10-25 21:02
8,2019-10-25 20:22,354 мин,Дмитровская,Царицыно,50 мин,стандарт плюс,2019-10-25 21:12
9,2019-10-25 20:28,360 мин,Дмитровская,Царицыно,55 мин,стандарт плюс,2019-10-25 21:23
